В результате экспериментов с моделью "Bag_of_Words" удалось добиться точности 0,94061 (это между 198 и 199 позициями в списке лидеров). Использовала Multinomial Naive Bayes классификатор и TfidfVectorizer со следующими параметрами: учет биграмм и триграмм, логарифмированные документные частоты (sublinear_tf=True), неограниченное количество признаков, максимальной документная частота  0,95 (из 1), минимальная документная частота 2. (Только Multinomial Naive Bayes и TfidfVectorizer уже дают 0.91261.)

Изначально, при следовании тьюториалу kaggle, точность была 0.84608, с добавлении биграмм (ngram_range = (1, 2)) -- 0.84632. Пробовала использовать стоп-слова из sklearn, но точность несколько понизилась -- 0.84102, стемминг, как оказалось, также не способствует повышению точности (про сложности лемматизации английских слов упоминала в первом задании). Также пробовала некоторые другие классификаторы (SVM, BernoulliNB, SGDLClassifier) с различными параметрами, наилучший результат был у SVM -- 0,9099.

Изменение коэффициента сглаживания (alpha в Multinomial Naive Bayes) и использование различных вариаций Ngramm при подсчтете tf-idf не позволяет достичь точности более 0,94061.

In [13]:
import re
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from bs4 import BeautifulSoup

# from nltk.stem.porter import *
# stemmer = PorterStemmer()
# def stem_tokens(tokens, stemmer):
#     stemmed = []
#     for item in tokens:
#         stemmed.append(stemmer.stem(item))
#     return stemmed

def review2words(raw_review):
    review_text = BeautifulSoup(raw_review, "lxml").get_text()
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    words = letters_only.lower().split()
    stops = set(stopwords.words("english"))
    meaningful_words = [w for w in words if not w in stops]
#    meaningful_words = stem_tokens(meaningful_words, stemmer)
    return " ".join(meaningful_words)

In [14]:
train = pd.read_csv("/home/anya/python/word2vec/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv('/home/anya/python/word2vec/testData.tsv', header=0, delimiter="\t", quoting=3)
clean_train_reviews = []
for i in range(train["review"].size):
    clean_train_reviews.append(review2words(train["review"][i]))
clean_test_reviews = []
for i in range(test["review"].size):
    clean_test_reviews.append(review2words(test["review"][i]))
print(len(clean_train_reviews), len(clean_test_reviews))

25000 25000


In [15]:
tfv = TfidfVectorizer(min_df=2, max_df=0.95, max_features=None, ngram_range = (1, 3), sublinear_tf=True)
vectorizer = tfv.fit(clean_train_reviews)
train_data_features = vectorizer.transform(clean_train_reviews)
test_data_features = vectorizer.transform(clean_test_reviews)

In [16]:
model = MultinomialNB(alpha=0.1)
model.fit(train_data_features, train["sentiment"])
result = model.predict_proba(test_data_features)[:,1]
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv('/home/anya/python/word2vec/Bag_of_Words_model.csv', index=False, quoting=3)